In [4]:
from dependencies import *

from final_data import final_df, path, day

'Loading Final DataFrame'


<ipython-input-4-740a86f4beec>:3: DtypeWarning: Columns (67,68,69,70,71) have mixed types.Specify dtype option on import or set low_memory=False.
  from final_data import final_df, path, day


In [ ]:
# original_df = pd.read_csv('data/final/2021-08-15.csv')
df = final_df.copy()
df

,date,fips,Lead (TSP) LC,Carbon monoxide,Sulfur dioxide,Nitrogen dioxide (NO2),Ozone,PM10 - LC,PM2.5 - Local Conditions,county,...,Administered_Dose1_Recip_18PlusPop_Pct,Administered_Dose1_Recip_65PlusPop_Pct,SVI_CTGY,Series_Complete_Pop_Pct_SVI,Series_Complete_12PlusPop_Pct_SVI,Series_Complete_18PlusPop_Pct_SVI,Series_Complete_65PlusPop_Pct_SVI,active,new,percentage_new
0,2020-03-22,36061,NaN,0.19075,NaN,NaN,0.034988,NaN,NaN,New York City,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-22,36059,NaN,NaN,0.20119,NaN,NaN,NaN,NaN,Nassau,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-22,36119,NaN,NaN,NaN,NaN,0.039600,NaN,NaN,Westchester,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-22,36103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Suffolk,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-22,36087,NaN,NaN,NaN,NaN,0.037064,NaN,NaN,Rockland,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1461152,2021-06-30,56037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sweetwater,...,42.6,80.4,Low-Mod,Low VC/Low-Mod SVI,Low-Mod VC/Low-Mod SVI,Low-Mod VC/Low-Mod SVI,High VC/Low-Mod SVI,107.0,NaN,NaN
1461153,2021-06-30,56039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Teton,...,90.9,87.0,Low,High VC/Low SVI,High VC/Low SVI,High VC/Low SVI,High VC/Low SVI,4.0,NaN,NaN
1461154,2021-06-30,56041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Uinta,...,46.0,76.3,Low-Mod,Low-Mod VC/Low-Mod SVI,Low-Mod VC/Low-Mod SVI,Mod-High VC/Low-Mod SVI,High VC/Low-Mod SVI,18.0,NaN,NaN
1461155,2021-06-30,56043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washakie,...,40.5,72.5,Low-Mod,Low VC/Low-Mod SVI,Low-Mod VC/Low-Mod SVI,Low-Mod VC/Low-Mod SVI,Mod-High VC/Low-Mod SVI,1.0,NaN,NaN


# Fill NaN Vaccination with 0

In [3]:
vaccine_cols = ['Series_Complete_Pop_Pct', 'Series_Complete_12PlusPop_Pct', 'Series_Complete_18PlusPop_Pct', 'Series_Complete_65PlusPop_Pct', 'Administered_Dose1_Pop_Pct', 'Administered_Dose1_Recip_12PlusPop_Pct', 'Administered_Dose1_Recip_18PlusPop_Pct', 'Administered_Dose1_Recip_65PlusPop_Pct']

df[vaccine_cols] = df[vaccine_cols].fillna(0)

# Calculate Active Cases

*contagious=14* - number of days where an infected individual is contagious

*(Confirmed - Death) - (Confirmed14 - Deaths14)*

In [4]:
contagious = 14

df['active'] = (df['Confirmed'] - df['Deaths']).fillna(0)
df['active'] = df.groupby('fips')['active'].diff(contagious)
# df.drop('Active', axis=1, inplace=True)


# Calculate New Cases
*average=7*

average of future *average* cases

In [5]:
average = 7

df['new'] = df.groupby('fips')['Confirmed'].diff(average).shift(-average) / average
df.loc[df['new'] < 0, 'new'] = np.nan

# Calculate New as a Percentage of Active

In [6]:
df['percentage_new'] = df['new'] / df['active'] * 100

# Interpolate Data

## For each county, take the average values of the surrounding counties for each day

In [ ]:
adjacency

In [7]:
adjacency = json.load(open('docs/county_adjacency.json'), object_hook=lambda x: {int(k): v for k, v in x.items()})

In [23]:
np.isnan(df['Carbon monoxide'].unique()[1])

True

In [13]:
df[df[pollutants].isnull().all(axis=1)]

,date,fips,lead,carbon monoxide,sulfur dioxide,nitrogen dioxide,ozone,PM10,PM2.5,county,...,Administered_Dose1_Recip_18PlusPop_Pct,Administered_Dose1_Recip_65PlusPop_Pct,SVI_CTGY,Series_Complete_Pop_Pct_SVI,Series_Complete_12PlusPop_Pct_SVI,Series_Complete_18PlusPop_Pct_SVI,Series_Complete_65PlusPop_Pct_SVI,active,new,percentage_new
3,2020-03-22,36103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Suffolk,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,2020-03-22,36093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Schenectady,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,2020-03-22,36083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rensselaer,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,2020-03-22,36111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ulster,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,2020-03-22,36021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Columbia,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1461152,2021-06-30,56037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sweetwater,...,42.6,80.4,Low-Mod,Low VC/Low-Mod SVI,Low-Mod VC/Low-Mod SVI,Low-Mod VC/Low-Mod SVI,High VC/Low-Mod SVI,107.0,NaN,NaN
1461153,2021-06-30,56039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Teton,...,90.9,87.0,Low,High VC/Low SVI,High VC/Low SVI,High VC/Low SVI,High VC/Low SVI,4.0,NaN,NaN
1461154,2021-06-30,56041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Uinta,...,46.0,76.3,Low-Mod,Low-Mod VC/Low-Mod SVI,Low-Mod VC/Low-Mod SVI,Mod-High VC/Low-Mod SVI,High VC/Low-Mod SVI,18.0,NaN,NaN
1461155,2021-06-30,56043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washakie,...,40.5,72.5,Low-Mod,Low VC/Low-Mod SVI,Low-Mod VC/Low-Mod SVI,Low-Mod VC/Low-Mod SVI,Mod-High VC/Low-Mod SVI,1.0,NaN,NaN


In [14]:
df = final_df.copy()

missing_df = df[df[pollutants].isnull().all(axis=1)]
for _, row in missing_df.iterrows():
	for pollutant in pollutants:
		row[pollutant] = np.nanmean([df[(df['fips'] == fips) & (df['date'] == row['date']), pollutant] for fips in adjacency[row['fips']]])


In [12]:
df

,Confirmed,date
0,9654,2020-03-22
1,1900,2020-03-22
2,1873,2020-03-22
3,1034,2020-03-22
4,455,2020-03-22
...,...,...
1461152,4771,2021-06-30
1461153,3801,2021-06-30
1461154,2320,2021-06-30
1461155,926,2021-06-30


# Export DataFrame

In [20]:
path = f'data/cleaned/{day}.csv'
df.to_csv(path, index=False)